In [2]:
import pandas as pd
import datetime
import numpy as np
import math
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(database="finances",
                        host="localhost",
                        user="postgres",
                        password="91228b0b-7c51-4671-a2f4-729a1837ded3",
                        port=5432)

In [211]:
cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

In [212]:
def split_cross_days(shifts):
    new_rows = []

    for row in shifts.to_dict(orient="records"):
        if row["start_time"].day < row["end_time"].day:
            new_range = pd.date_range(row["start_time"],x["end_time"],freq="min")
            days = np.unique(new_range.day).tolist()
            for day in days:
                filtered_times = new_range[new_range.day == day]
                new_row = {"start_time":filtered_times[0],"end_time":filtered_times[-1],"employer":row["employer"]}
                new_rows.append(new_row)
        else:
            new_row = {"start_time":row["start_time"],"end_time":row["end_time"],"employer":row["employer"]}
            new_rows.append(new_row)
            
    return pd.DataFrame(new_rows)

In [213]:
def get_shifts(username,year,month):
    command = """
    select start_time,end_time,employer from shifts 
    join contracts on contracts.contract_id = shifts.contract_id
    join users on contracts.user_id = contracts.user_id
    where 
    date_part('year',start_time) = %s and date_part('month',start_time) = %s or
    date_part('year',end_time) = %s and date_part('month',end_time) = %s
    and users.email = %s
    order by start_time asc;"""
    params = (year,month,year,month,username)
    cursor.execute(command,params)
    data = cursor.fetchall()
    return data

In [214]:
def merge_shifts(shifts,days,year,month):
    begin = pd.Timestamp(year=year, month=month, day=1, hour=0)
    end = begin + pd.offsets.MonthEnd() + pd.DateOffset(hour=23) + pd.DateOffset(minutes=59)
    begin_f, end_f = begin.strftime("%Y-%m-%d %H:%M"), end.strftime("%Y-%m-%d %H:%M")
    shifts.loc[shifts['start_time'] < begin_f ,'start_time'] = begin_f
    shifts.loc[shifts['end_time'] >= end_f, 'end_time'] = end_f
    shifts["date"] = shifts["start_time"].dt.date.astype(str)
    shifts["start"] = shifts["start_time"].dt.time.astype(str).str.slice(start=0,stop=5)
    shifts["end"] = shifts["end_time"].dt.time.astype(str).str.slice(start=0,stop=5)
    calendar_f = pd.DataFrame(data={"date":days}).astype(str)
    merged = pd.merge(calendar_f, shifts.drop(columns=["start_time","end_time"]), on="date", how="outer")
    return merged
    #return shifts

In [219]:
def create_calendar(year,month):
    normalized = pd.Timestamp(year=year,month=month,day=1)
    european_day_of_week = normalized.day_of_week
    if european_day_of_week != 6:
        first_cal_day = normalized - pd.Timedelta(european_day_of_week + 1, unit="d")
    else:
        first_cal_day = normalized
    last_day_of_month = pd.Timestamp(year=normalized.year,month=normalized.month,day=normalized.daysinmonth)
    day_diff = (last_day_of_month - first_cal_day).days * 1 + 1
    num_days = math.ceil(day_diff / 7) * 7
    last_cal_day =first_cal_day + pd.Timedelta(num_days-1, unit="d")
    days = pd.date_range(first_cal_day,last_cal_day)
    shift_data = get_shifts("koji.gabriel218@gmail.com",year,month)
    shifts = pd.DataFrame(shift_data)
    print(shifts)

    if any((shifts["start_time"].dt.day < shifts["end_time"].dt.day).tolist()):
        shifts = split_cross_days(shifts)
        print(shifts)
    merged = merge_shifts(shifts,days,year,month)
    
    days = {}

    for row in merged.to_dict(orient="records"):
        if row["date"] not in days:
           days[row["date"]] = []
        
        if not pd.isnull(row["employer"]):
            shift = {"employer":row["employer"],"start":row["start"],"end":row["end"]}
            days[row["date"]].append(shift)
            
    calendar = [{"day":day,"shifts":days[day]} for day in days]
    return calendar
    #return shifts

In [220]:
calendar = create_calendar(2024,5)

                 start_time                  end_time  employer
0 2024-04-30 20:00:00+03:00 2024-05-01 04:00:00+03:00  S-Market
1 2024-05-14 08:00:00+03:00 2024-05-14 16:00:00+03:00     Ikeaa
2 2024-05-14 17:00:00+03:00 2024-05-14 20:00:00+03:00  S-Market
3 2024-05-17 20:00:00+03:00 2024-05-18 05:00:00+03:00     Ikeaa
4 2024-05-19 23:00:00+03:00 2024-05-21 07:00:00+03:00     Ikeaa
5 2024-05-23 08:00:00+03:00 2024-05-23 16:00:00+03:00     Ikeaa
6 2024-05-31 20:00:00+03:00 2024-06-01 04:00:00+03:00     Ikeaa
                  start_time                  end_time  employer
0  2024-04-30 20:00:00+03:00 2024-05-01 04:00:00+03:00  S-Market
1  2024-05-14 08:00:00+03:00 2024-05-14 16:00:00+03:00     Ikeaa
2  2024-05-14 17:00:00+03:00 2024-05-14 20:00:00+03:00  S-Market
3  2024-06-01 00:00:00+03:00 2024-06-01 04:00:00+03:00     Ikeaa
4  2024-05-17 20:00:00+03:00 2024-05-17 23:59:00+03:00     Ikeaa
5  2024-05-18 00:00:00+03:00 2024-05-18 23:59:00+03:00     Ikeaa
6  2024-05-19 00:00:00+03:00 2024

In [197]:
any((calendar["start_time"].dt.day < calendar["end_time"].dt.day).tolist())

True

In [191]:
new_rows = []

for x in calendar.to_dict(orient="records"):
    if x["start_time"].day < x["end_time"].day:
        new_range = pd.date_range(x["start_time"],x["end_time"],freq="min")
        days = np.unique(new_range.day).tolist()
        for day in days:
            filtered_times = new_range[new_range.day == day]
            new_row = {"start_time":filtered_times[0],"end_time":filtered_times[-1],"employer":x["employer"]}
            new_rows.append(new_row)
    else:
        new_row = {"start_time":x["start_time"],"end_time":x["end_time"],"employer":x["employer"]}
        new_rows.append(new_row)

In [192]:
calendar

,start_time,end_time,employer
0,2024-04-30 20:00:00+03:00,2024-05-01 04:00:00+03:00,S-Market
1,2024-05-14 08:00:00+03:00,2024-05-14 16:00:00+03:00,Ikeaa
2,2024-05-14 17:00:00+03:00,2024-05-14 20:00:00+03:00,S-Market
3,2024-05-17 20:00:00+03:00,2024-05-18 05:00:00+03:00,Ikeaa
4,2024-05-19 23:00:00+03:00,2024-05-21 07:00:00+03:00,Ikeaa
5,2024-05-23 08:00:00+03:00,2024-05-23 16:00:00+03:00,Ikeaa
6,2024-05-31 20:00:00+03:00,2024-06-01 04:00:00+03:00,Ikeaa


In [193]:
pd.DataFrame(new_rows)

,start_time,end_time,employer
0,2024-04-30 20:00:00+03:00,2024-05-01 04:00:00+03:00,S-Market
1,2024-05-14 08:00:00+03:00,2024-05-14 16:00:00+03:00,Ikeaa
2,2024-05-14 17:00:00+03:00,2024-05-14 20:00:00+03:00,S-Market
3,2024-05-17 20:00:00+03:00,2024-05-17 23:59:00+03:00,Ikeaa
4,2024-05-18 00:00:00+03:00,2024-05-18 05:00:00+03:00,Ikeaa
5,2024-05-19 23:00:00+03:00,2024-05-19 23:59:00+03:00,Ikeaa
6,2024-05-20 00:00:00+03:00,2024-05-20 23:59:00+03:00,Ikeaa
7,2024-05-21 00:00:00+03:00,2024-05-21 07:00:00+03:00,Ikeaa
8,2024-05-23 08:00:00+03:00,2024-05-23 16:00:00+03:00,Ikeaa
9,2024-05-31 20:00:00+03:00,2024-06-01 04:00:00+03:00,Ikeaa
